In [1]:
from simulation import getHpen, getU, blocks2Mat
import numpy as np
import pennylane as qml
from itertools import product
from fractions import Fraction

In [2]:
# helper functions

def get_P0_and_HpenInverse(Hpen):
    """
    get `P0` (projector onto the nullspace of `Hpen`) and `HpenInverse` (pseudoinverse of `Hpen`).

    Make sure that the eigenvalues of `Hpen` are all integers!
    """
    e, u = np.linalg.eigh(Hpen)

    a = np.array([1 if abs(x) < 0.5 else 0 for x in e])
    P0 = u @ (np.expand_dims(a, axis=1) * u.conj().T)

    b = np.array([1/x if abs(x) > 0.5 else 0 for x in e])
    HpenInverse = u @ (np.expand_dims(b, axis=1) * u.conj().T)

    return P0, HpenInverse

# Inner-block logical operators

In [3]:
blocksize = 4
n_block = 1
n_phys = blocksize * n_block
n_logi = n_phys // 2

Hpen = getHpen(n_phys, n_block) # penalty hamiltonian
P0, HpenInverse = get_P0_and_HpenInverse(Hpen) # projector onto the nullspace of Hpen, and pseudoinverse of Hpen
U = getU(blocksize, n_block) # encoder isometry, i.e., the i-th column is the i-th logical basis state
Udag = U.conj().T
Penc = U @ Udag # projector onto the encoding subspace

assert np.allclose(Hpen @ P0, 0)
assert np.allclose(HpenInverse @ P0, 0)
assert np.allclose(Hpen @ HpenInverse, np.eye(2**n_phys) - P0)
assert np.allclose(Udag @ U, np.eye(2**n_logi))
assert np.allclose(Penc, P0)


In [9]:
def check_inner_block_1st_order_logical_operator(Henc1, precision=5):
    # check Henc1 is block diagonal w.r.t. P0 and I-P0
    if not np.allclose((np.eye(2**n_phys) - P0) @ Henc1 @ P0, 0):
        return "Fail", ""

    # express logical operator in the logical pauli basis
    Hlogi = Udag @ Henc1 @ U
    Hlogi_decomposed = qml.pauli_decompose(Hlogi, hide_identity=True, pauli=True)
    logi_expr = ""
    for i, (term, coeff) in enumerate(Hlogi_decomposed.items()):
        coeff = coeff.real
        term = str(term).replace("@", "*").replace(" ", "").replace("(", "").replace(")", "")
        if coeff > 0 and i > 0:
            logi_expr += "+"
        logi_expr += f"{coeff:.{precision}f}*{term}"
    return "Pass", logi_expr

In [11]:
paulis = ["X", "Y", "Z"]
with open("enumerate_inner_block_1st_order_logical_ops.txt", "w") as file:
    file.write("Henc1 -> Htar\n")
    for a, b in product(paulis, repeat=2):
        print(f"Checking a, b = {a}, {b}")
        for i in range(4):
            for j in range(i+1, 4):
                Henc1_expr = f'{a}{i}*{b}{j}'
                Henc1 = blocks2Mat(n_phys, [(1, Henc1_expr)])
                msg, logi_expr = check_inner_block_1st_order_logical_operator(Henc1)
                if msg == "Pass":
                    file.write(f"{Henc1_expr} -> {logi_expr}\n")

Checking a, b = X, X
Checking a, b = X, Y
Checking a, b = X, Z
Checking a, b = Y, X
Checking a, b = Y, Y
Checking a, b = Y, Z
Checking a, b = Z, X
Checking a, b = Z, Y
Checking a, b = Z, Z


In [20]:
def check_inner_block_2nd_order_logical_operator(Henc2, precision=5):
    # check Henc2 is zero up to 1st order perturbation
    if not np.allclose(P0 @ Henc2 @ P0, 0):
        return "Fail1", ""

    # express logical operator in the logical pauli basis
    Hlogi = - Udag @ Henc2 @ HpenInverse @ Henc2 @ U
    if np.allclose(Hlogi, 0):
        return "Fail2", ""

    Hlogi_decomposed = qml.pauli_decompose(Hlogi, hide_identity=True, pauli=True)
    logi_expr = ""
    for i, (term, coeff) in enumerate(Hlogi_decomposed.items()):
        coeff = coeff.real
        term = str(term).replace("@", "*").replace(" ", "").replace("(", "").replace(")", "")
        if coeff > 0 and i > 0:
            logi_expr += "+"
        logi_expr += f"{coeff:.{precision}f}*{term}"
    return "Pass", logi_expr

In [21]:
paulis = ["X", "Y", "Z"]
with open("enumerate_inner_block_2nd_order_logical_ops.txt", "w") as file:
    file.write("Henc2 -> Htar\n")
    # search for single-qubit operators
    for a in paulis:
        print(f"Checking a = {a}")
        for i in range(4):
            Henc2_expr = f'{a}{i}'
            Henc2 = blocks2Mat(n_phys, [(1, Henc2_expr)])
            msg, logi_expr = check_inner_block_2nd_order_logical_operator(Henc2)
            if msg == "Pass":
                file.write(f"{Henc2_expr} -> {logi_expr}\n")
    # search for two-qubit operators
    for a, b in product(paulis, repeat=2):
        print(f"Checking a, b = {a}, {b}")
        for i in range(4):
            for j in range(i+1, 4):
                Henc2_expr = f'{a}{i}*{b}{j}'
                Henc2 = blocks2Mat(n_phys, [(1, Henc2_expr)])
                msg, logi_expr = check_inner_block_2nd_order_logical_operator(Henc2)
                if msg == "Pass":
                    file.write(f"{Henc2_expr} -> {logi_expr}\n")
    # search for sum of two two-qubit operators
    for a, b, c, d in product(paulis, repeat=4):
        print(f"Checking a, b, c, d = {a}, {b}, {c}, {d}")
        for i in range(4):
            for j in range(i+1, 4):
                for k in range(4):
                    for l in range(k+1, 4):
                        if i > k:
                            continue
                        if i == k and j > l:
                            continue
                        if i == k and j == l and a == c and b == d:
                            continue
                        Henc2_expr = f'{a}{i}*{b}{j}+{c}{k}*{d}{l}'
                        Henc2 = blocks2Mat(n_phys, [(1, Henc2_expr)])
                        msg, logi_expr = check_inner_block_2nd_order_logical_operator(Henc2)
                        if msg == "Pass":
                            file.write(f"{Henc2_expr} -> {logi_expr}\n")

Checking a = X
Checking a = Y
Checking a = Z
Checking a, b = X, X
Checking a, b = X, Y
Checking a, b = X, Z
Checking a, b = Y, X
Checking a, b = Y, Y
Checking a, b = Y, Z
Checking a, b = Z, X
Checking a, b = Z, Y
Checking a, b = Z, Z
Checking a, b, c, d = X, X, X, X
Checking a, b, c, d = X, X, X, Y
Checking a, b, c, d = X, X, X, Z
Checking a, b, c, d = X, X, Y, X
Checking a, b, c, d = X, X, Y, Y
Checking a, b, c, d = X, X, Y, Z
Checking a, b, c, d = X, X, Z, X
Checking a, b, c, d = X, X, Z, Y
Checking a, b, c, d = X, X, Z, Z
Checking a, b, c, d = X, Y, X, X
Checking a, b, c, d = X, Y, X, Y
Checking a, b, c, d = X, Y, X, Z
Checking a, b, c, d = X, Y, Y, X
Checking a, b, c, d = X, Y, Y, Y
Checking a, b, c, d = X, Y, Y, Z
Checking a, b, c, d = X, Y, Z, X
Checking a, b, c, d = X, Y, Z, Y
Checking a, b, c, d = X, Y, Z, Z
Checking a, b, c, d = X, Z, X, X
Checking a, b, c, d = X, Z, X, Y
Checking a, b, c, d = X, Z, X, Z
Checking a, b, c, d = X, Z, Y, X
Checking a, b, c, d = X, Z, Y, Y
Checkin

# Cross-block logical operators

In [3]:
blocksize = 4
n_block = 2
n_phys = blocksize * n_block
n_logi = n_phys // 2

Hpen = getHpen(n_phys, n_block) # penalty hamiltonian
P0, HpenInverse = get_P0_and_HpenInverse(Hpen) # projector onto the nullspace of Hpen, and pseudoinverse of Hpen
U = getU(blocksize, n_block) # encoder isometry, i.e., the i-th column is the i-th logical basis state
Udag = U.conj().T
Penc = U @ Udag # projector onto the encoding subspace

assert np.allclose(Hpen @ P0, 0)
assert np.allclose(HpenInverse @ P0, 0)
assert np.allclose(Hpen @ HpenInverse, np.eye(2**n_phys) - P0)
assert np.allclose(Udag @ U, np.eye(2**n_logi))
assert np.allclose(Penc @ P0, Penc)


In [4]:
def check_logical_operator(Henc2, precision=5):
    # check Henc2 is zero up to 1st order perturbation
    if not np.allclose(P0 @ Henc2 @ P0, 0):
        return "Fail1", ""
    
    A = - Henc2 @ HpenInverse @ Henc2
    # Heff = P0 @ A @ P0 is the effective hamiltonian up to 2nd order

    # check Heff is block diagonal w.r.t. Penc and P0-Penc
    if not np.allclose((P0 - Penc) @ A @ Penc, 0):
        return "Fail2", ""

    # express logical operator in the logical pauli basis
    Hlogi = Udag @ A @ U
    Hlogi_decomposed = qml.pauli_decompose(Hlogi, hide_identity=True, pauli=True)
    logi_expr = ""
    for i, (term, coeff) in enumerate(Hlogi_decomposed.items()):
        coeff = coeff.real
        term = str(term).replace("@", "*").replace(" ", "").replace("(", "").replace(")", "")
        if coeff > 0 and i > 0:
            logi_expr += "+"
        logi_expr += f"{coeff:.{precision}f}*{term}"
    return "Pass", logi_expr

In [ ]:
paulis = ["X", "Y", "Z"]
with open("enumerate_cross_block_2nd_order_logical_ops.txt", "w") as file:
    for a, b in product(paulis, repeat=2):
        print(f"Checking a, b = {a}, {b}")
        for k in range(4, 8):
            for i, j in [(0, 2), (0, 3), (1, 2), (1, 3)]:
                Henc2_expr = f'{a}{i}*{b}{k}+{a}{j}*{b}{k}'
                Henc2 = blocks2Mat(n_phys, [(1, Henc2_expr)])
                msg, logi_expr = check_logical_operator(Henc2)
                if msg == "Pass":
                    file.write(f"{Henc2_expr} -> {logi_expr}\n")
    # for a, b, c in product(paulis, repeat=3):
    #     print(f"Checking a, b, c = {a}, {b}, {c}")
    #     for k in range(4, 8):
    #         for l in range(k, 8):
    #             for i, j in [(0, 2), (0, 3), (1, 2), (1, 3)]:
    #                 Henc2_expr = f'{a}{i}*{b}{k}+{a}{j}*{b}{k}+{a}{i}*{c}{l}+{a}{j}*{c}{l}'
    #                 Henc2 = blocks2Mat(n_phys, [(1, Henc2_expr)])
    #                 msg, logi_expr = check_logical_operator(Henc2)
    #                 if msg == "Pass":
    #                     file.write(f"{Henc2_expr} -> {logi_expr}\n")

Checking a, b = X, X
Checking a, b = X, Y
Checking a, b = X, Z
Checking a, b = Y, X
Checking a, b = Y, Y
Checking a, b = Y, Z
Checking a, b = Z, X
Checking a, b = Z, Y
Checking a, b = Z, Z


# Draft

In [6]:
check_logical_operator(blocks2Mat(n_phys, [(1, "X0*Y4+X2*Y4")]), precision=15)

('Pass',
 '0.485714285714285*X3+0.085714285714286*Z2-0.485714285714285*X0*X3-0.085714285714286*X0*Z2')

In [7]:
Fraction(0.485714285714285).limit_denominator(1000)

Fraction(17, 35)

In [8]:
Henc2 = blocks2Mat(n_phys, [(np.sqrt(35/3), "X0*Y4+X2*Y4"), (np.sqrt(136/3), "X0*Z4+X2*Z4")])
check_logical_operator(Henc2, precision=10)

('Pass', '1.0000000000*Z2-1.0000000000*X0*Z2')